In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

import sys
sys.path.append('..')
# Code provide to you for training and evaluation
from utils import train, evaluate, set_seed_nb, unit_test_values, deterministic_init
from models import VanillaRNN

ImportError: cannot import name 'train' from 'utils' (/home/louis/.local/lib/python3.10/site-packages/utils/__init__.py)

In [ ]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("You are using device: %s" % device)